  error: subprocess-exited-with-error
  
  × python setup.py egg_info did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  Preparing metadata (setup.py) ... error
error: metadata-generation-failed

× Encountered error while generating package metadata.
╰─> See above for output.

note: This is an issue with the package mentioned above, not pip.
hint: See above for details.


In [ ]:
import tensorflow as tf
import os

In [ ]:
gpus = tf.config.experimental.list_physical_devices('GPU')
for gpu in gpus:
  tf.config.experimental.set_memory_growth(gpu,True)
len(gpus)

1

In [ ]:
import cv2
import imghdr

In [ ]:
data_dir= '/content/drive/MyDrive/data'

In [ ]:
image_exts = ['jpeg','jpg','bmp','png']


In [ ]:
happy_images = os.listdir(os.path.join(data_dir,'happy'))
sad_images = os.listdir(os.path.join(data_dir,'sad'))

In [ ]:
for image_class in os.listdir(data_dir):
  for image in os.listdir(os.path.join(data_dir,image_class)):
    image_path = os.path.join(data_dir,image_class,image)
    try:
      img = cv2.imread(image_path)
      print(img.shape)
      tip = imghdr.what(image_path)
      if tip not in image_exts:
        print("Image not in the correct format")
        os.remove(image_path)
    except Exception as e:
      print("Issue with the image")


In [ ]:
import numpy as np
import matplotlib.pyplot as plt

tensorflow.python.data.ops.dataset_ops.DatasetV2

In [ ]:
data = tf.keras.utils.image_dataset_from_directory(data_dir)

Found 305 files belonging to 2 classes.


In [ ]:
for d in data:
  print(d)

In [ ]:
data_iter = data.as_numpy_iterator()